In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from random import *
import itertools 
from scipy.optimize import minimize
from sklearn.metrics import r2_score

In [ ]:
def softmax(beta, value):   
    num = np.exp(value * beta)
    den = np.exp(value * beta).sum()    
    return num / den

In [ ]:
class parameter_fit(object):
    def __init__(self, df, bounds = ((0,1), (0,5)), guess = [0.1,1]):
        self.df = df
        self.bounds = bounds # range for alpha and beta
        self.guess = guess  # guess to aid scipy minimize
        
    def negative_log_likelihood(self, parameter):
        df = self.df
        alpha = parameter[0] # defining separate parameters
        beta = parameter[1]
        value = 0.5 * np.ones(2) # ensuring computes values for both levers/choices. Value starts at 0.5
        
        choices, rewards = df['right_or_left'].values.astype(int), df['reward'].values.astype(int)
        prob_log = 0
        session = df['session'].values.tolist()
        switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
        
        
        for choice, reward in zip(choices, rewards):
            if choice not in switch:
                value[choice] += alpha * (reward - value[choice])  
            else:
                value = 0.5
            prob_log += np.log(softmax(value, beta)[choice])

        return -prob_log

    def minimisation(self):
        bounds = (self.bounds)
        mini = minimize(self.negative_log_likelihood, self.guess,
                     method='L-BFGS-B',
                     bounds=bounds)  # method is optimisation algorithm,
        return mini

    def data_fit(self):
        data_fit = pd.DataFrame()
        for mice in self.df['mouse'].unique().tolist():
            df1 = self.df[self.df['mouse'] == mice]
            df1 = df1.reset_index(drop = True)
            mouse = df1['mouse'].unique().tolist()
            sim = parameter_fit(df1,self.bounds,self.guess)
            output = sim.minimisation()
            data = pd.DataFrame({'mouse':mouse,'alpha':output.x[0],'beta':output.x[1],
                                '- log likelihood':output.fun})
            data_fit = data_fit.append(data)
        return data_fit

In [ ]:
input_df = pd.read_pickle('dlight_data.pkl')
data = input_df

In [ ]:
session = data['session'].values.tolist() 
switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
switch.append(len(session))
block = list(np.diff(switch))
block = [451] + block

In [ ]:
name = [i for i in range(len(block))]
temp = []
for i, x in enumerate(block):
    t = [name[i] for a in range(block[i])]
    temp.append(t)
flat_list = [item for sublist in temp for item in sublist]

In [ ]:
data['mouse'] = flat_list

In [ ]:
model = parameter_fit(data)
fit = model.data_fit()

In [ ]:
def RW_fit(alpha, left_value, right_value, reward, choices):
    if choices == 1: #if right choice
        delta = (reward - right_value)
        right_value += alpha * (reward - right_value)  
        left_value = left_value
    elif choices == 0: #if left choice
        delta = (reward - left_value)
        left_value += alpha * (reward - left_value)     
        right_value = right_value
    return delta, left_value, right_value

In [ ]:
def infer(mouse):  #calulating representations of value and rpe for each session
    leftlever = 0.5
    rightlever = 0.5
    delta = []
    left_value = [0.5]
    right_value = [0.5]
    
    alpha = fit[fit['mouse']==mouse]['alpha'].item() 
    choices = data[data['mouse']==mouse]['right_or_left'].values.tolist() 
    session = data[data['mouse']==mouse]['session'].values.tolist() 
    reward = data[data['mouse']==mouse]['reward'].values.tolist()
    signal = data[data['mouse']==mouse]['cs_signal'].values.tolist()
    switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
    
    for index, lr in enumerate(reward):  
        if index in switch:
            leftlever = 0.5
            rightlever = 0.5
            a, leftlever, rightlever = RW_fit(alpha, leftlever, rightlever, reward[index], choices[index])
            delta.append(a)
            left_value.append(leftlever)
            right_value.append(rightlever)
        else:
            a, leftlever, rightlever = RW_fit(alpha, leftlever, rightlever, reward[index], choices[index])
            delta.append(a)
            left_value.append(leftlever)
            right_value.append(rightlever)
    
    return delta, left_value, right_value, reward, signal, switch

In [ ]:
delta, left_value, right_value, reward, signal, switch = infer(12)

In [ ]:
plt.plot(signal[100]) #example dopamine trace
sns.despine(left=True)
ax = plt.gca()
ax.axes.yaxis.set_visible(False)
ax.set(ylim=(-1.2, 2.8))
plt.xlabel('Frequency / 500 Hz', fontsize = 11)
ax.vlines([1500], -1.18, 2.8, linestyles='dashed', colors='k')
ax.vlines([1000], -1.18, 2.8, linestyles='dashed', colors='k')
ax.vlines([3000], -1.18, 2.8, linestyles='dashed', colors='k')
plt.savefig("5b.png", transparent=True, bbox_inches='tight')

In [ ]:
negRPE = []
noRPE = []
posRPE = []
for index, r in enumerate(delta):
    if delta[index] <= -0.25:
        negRPE.append(index)
    elif -0.25 < delta[index]< 0.25:
        noRPE.append(index)
    elif delta[index] >= 0.25:
        posRPE.append(index)
    else:
        pass

In [ ]:
negRPE_signal = []
noRPE_signal = []
posRPE_signal = []
for index, r in enumerate(signal):
    if index in negRPE:
        negRPE_signal.append(signal[index])
    elif index in noRPE:
        noRPE_signal.append(signal[index])
    elif index in posRPE:
        posRPE_signal.append(signal[index])

In [ ]:
from scipy import stats
def tsplot(ax, data, color, lw = 1,**kw): #making exemplar dopamine traces with error for different RPE ranges
    x = np.arange(data.shape[1])
    est = np.nanmean(data, axis=0)
    sd = np.nanstd(data, axis=0)/np.sqrt(data.shape[0])
    cis = (est - sd, est + sd)
    plt.fill_between(x,cis[0],cis[1],alpha=0.2, facecolor = color, **kw)
    plt.plot(x,est,**kw, c = color, linewidth = lw)

In [ ]:
ax = plt.figure(figsize = (8,3))
plt.subplot()
plot_data_cs = [(np.array(x) - np.mean(x[1000:1500])) for x in noRPE_signal]
tsplot(ax, np.array(plot_data_cs), color = 'k')
plt.axis('off')
plt.ylim(-0.8, 1.7)
plt.xlim(1000,3000)
plt.axhline(y = 0, linestyle = '--', c = '0.5')
plt.axvline(x = 1500, linestyle = '--', c = '0.5')
plt.savefig("noRPE.png", transparent=True, bbox_inches='tight')

In [ ]:
ax = plt.figure(figsize = (8,3))
plt.subplot()
plot_data_cs = [(np.array(x) - np.mean(x[1000:1500])) for x in posRPE_signal]
tsplot(ax, np.array(plot_data_cs), color = 'k')
plt.axis('off')
plt.ylim(-0.8, 1.7)
plt.xlim(1000,3000)
plt.axhline(y = 0, linestyle = '--', c = '0.5')
plt.axvline(x = 1500, linestyle = '--', c = '0.5')
plt.savefig("posRPE.png", transparent=True, bbox_inches='tight')

In [ ]:
ax = plt.figure(figsize = (8,3))
plt.subplot()
plot_data_cs = [(np.array(x) - np.mean(x[1000:1500])) for x in negRPE_signal]
tsplot(ax, np.array(plot_data_cs), color = 'k')
plt.axis('off')
plt.ylim(-0.8, 1.7)
plt.xlim(1000,3000)
plt.axhline(y = 0, linestyle = '--', c = '0.5')
plt.axvline(x = 1500, linestyle = '--', c = '0.5')
plt.savefig("negRPE.png", transparent=True, bbox_inches='tight')

In [ ]:
delta = []
signal = []
for n in np.arange(0,43,1): #calculating representations for each mouse
    a, left_value, right_value, reward, b, switch = infer(n)
    delta.append(a)
    signal.append(b)
delta = np.concatenate(delta)
signal = np.concatenate(signal)

In [ ]:
negRPE = []
noRPE = []
posRPE = []
for index, r in enumerate(delta):
    if delta[index] <= -0.25:
        negRPE.append(index)
    elif -0.25 <= delta[index]<= 0.25:
        noRPE.append(index)
    elif delta[index] >= 0.25:
        posRPE.append(index)
    else:
        pass

In [ ]:
negRPE_signal = []
noRPE_signal = []
posRPE_signal = []
for index, r in enumerate(signal):
    if index in negRPE:
        negRPE_signal.append(signal[index])
    elif index in noRPE:
        noRPE_signal.append(signal[index])
    elif index in posRPE:
        posRPE_signal.append(signal[index])

In [ ]:
negRPEsummary = []
for i, j in enumerate(negRPE_signal):
    f = np.mean(negRPE_signal[i][1500:3000]) - np.mean(negRPE_signal[i][1000:1500])
    negRPEsummary.append(f)
    
noRPEsummary = []
for i, j in enumerate(noRPE_signal):
    g = np.mean(noRPE_signal[i][1500:3000]) - np.mean(noRPE_signal[i][1000:1500])
    noRPEsummary.append(g)
    
posRPEsummary = []
for i, j in enumerate(posRPE_signal):
    h = np.mean(posRPE_signal[i][1500:3000]) - np.mean(posRPE_signal[i][1000:1500])
    posRPEsummary.append(h)

In [ ]:
def mean_confidence_interval(data, confidence=0.95):  #used to calculate points for 5c
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h